In [86]:
import spacy
import pandas as pd
from spacy.training.example import Example
from spacy.tokens import DocBin
from tqdm import tqdm

In [87]:
!pip install -q spacy
!python -m spacy download -q en_core_web_sm


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



✘ No compatible package found for '-q' (spaCy v3.7.2)



In [88]:
df = pd.read_csv('atis_train.csv')
nlp = spacy.load("en_core_web_sm")
df.head(5)

,id,intent,text,slots
0,0,flight,i want to fly from boston at 838 am and arrive...,O O O O O B-fromloc.city_name O B-depart_time....
1,1,flight,what flights are available from pittsburgh to ...,O O O O O B-fromloc.city_name O B-toloc.city_n...
2,2,flight_time,what is the arrival time in san francisco for ...,O O O B-flight_time I-flight_time O B-fromloc....
3,3,airfare,cheapest airfare from tacoma to orlando,B-cost_relative O O B-fromloc.city_name O B-to...
4,4,airfare,round trip fares from pittsburgh to philadelph...,B-round_trip I-round_trip O O B-fromloc.city_n...


In [89]:
df['text'].head(5)

0    i want to fly from boston at 838 am and arrive...
1    what flights are available from pittsburgh to ...
2    what is the arrival time in san francisco for ...
3              cheapest airfare from tacoma to orlando
4    round trip fares from pittsburgh to philadelph...
Name: text, dtype: object

In [90]:

for x in range(0,len(df['text'])):
    doc = nlp(df['text'][x])
    ents = []
    #for start, end, label in __annotations__:
     #   span = doc.char_span(start, end, label=label)
     #   ents.append(span)
    #doc.ents = ents
    #df.add(doc)


In [91]:
# Prepare training data
df['tokens'] = df['text'].apply(lambda x: [token.text for token in nlp(x)])
train_data = []

for index, row in df.iterrows():
    text = row['text']
    entities = row['slots'].split()  # Assuming 'slots' contains entity annotations
    tokens = row['tokens']

    # Convert entity annotations to spaCy format
    annotations = {'entities': []}
    start = 0

    for entity in entities:
        if entity.startswith('B-'):
            label = entity.split('-')[1]
            end = start + 1

            # Check if the end index is within the bounds of the 'tokens' list
            if end <= len(tokens):
                annotations['entities'].append((start, end, label))
            else:
                print(f"Warning: Entity annotation exceeds token length in '{text}'")

            start = end

    train_data.append((text, annotations))

# Train the spaCy NER model
ner = nlp.get_pipe("ner")

# Add labels to the NER model
for _, annotations in train_data:
    for ent in annotations.get('entities', []):
        ner.add_label(ent[2])


In [95]:
train_data[0:2]

[('i want to fly from boston at 838 am and arrive in denver at 1110 in the morning',
  {'entities': [(0, 1, 'fromloc.city_name'),
    (1, 2, 'depart_time.time'),
    (2, 3, 'toloc.city_name'),
    (3, 4, 'arrive_time.time'),
    (4, 5, 'arrive_time.period_of_day')]}),
 ('what flights are available from pittsburgh to baltimore on thursday morning',
  {'entities': [(0, 1, 'fromloc.city_name'),
    (1, 2, 'toloc.city_name'),
    (2, 3, 'depart_date.day_name'),
    (3, 4, 'depart_time.period_of_day')]})]

In [93]:
#Preparing Training Data to train.spacy
db = DocBin() # create a DocBin object

for text, annot in tqdm(train_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            continue
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object

 10%|▉         | 495/4978 [00:00<00:00, 4922.32it/s]

100%|██████████| 4978/4978 [00:00<00:00, 6910.37it/s]


In [94]:
python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy


SyntaxError: invalid syntax (3541811700.py, line 1)